<a href="https://colab.research.google.com/github/guimauve007/MPPT-builder/blob/main/mppt_builder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>